In [ ]:
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath('./../src'))

import IRBEM
import data_loader
import datetime
from cdflib.epochs_astropy import CDFAstropy as cdfepoch
import numpy as np
import chorus_machine_learning_helper
import tqdm

In [2]:
start_time = datetime.datetime(year = 1998, month = 1, day = 1, hour = 1, minute = 1, second = 1)
end_time = datetime.datetime(year = 1999, month = 1, day = 1, hour = 1, minute = 1, second = 1)

In [4]:
year = 1998
start_of_year_UNIX_TIME = datetime.datetime(year = year, month = 1, day = 1).timestamp()
end_of_year_UNIX_TIME = datetime.datetime(year = year + 1, month = 1, day = 1).timestamp()
equally_spaced_minutes = np.arange(start = start_of_year_UNIX_TIME, stop = end_of_year_UNIX_TIME + 60, step = 60)

MPE = chorus_machine_learning_helper.load_MPE_year(year)

Finished loading POES data for year : 1998


In [5]:
sat = MPE[0]

unix_times_of_averages = []
avg_geog_lat = []
avg_geog_lon = []
L_avg = []

for MINUTE in equally_spaced_minutes:
    
    TIME_RANGE = np.searchsorted(a = sat["UNIX_TIME"], v = [(MINUTE - 30), (MINUTE + 30)])
    
    if (TIME_RANGE[0] != TIME_RANGE[1]):
        
        unix_times_of_averages.append(MINUTE)    
        avg_geog_lat.append(np.nanmean(sat["geogLat"][TIME_RANGE[0] : TIME_RANGE[1]]))
        y_of_lon = np.nanmean(np.sin(np.deg2rad(sat["geogLon"][TIME_RANGE[0] : TIME_RANGE[1]])))
        x_of_lon = np.nanmean(np.cos(np.deg2rad(sat["geogLon"][TIME_RANGE[0] : TIME_RANGE[1]])))            
        avg_geog_lon.append(np.mod((np.rad2deg(np.arctan2(y_of_lon, x_of_lon)) + 360), 360))
        L_avg.append(np.nanmean(sat["lValue"][TIME_RANGE[0] : TIME_RANGE[1]]))

In [6]:
KP_INTERPOLATED = np.interp(unix_times_of_averages, xp = time, fp = Kp, left=np.nan, right=np.nan).astype(np.float64)

In [9]:
points = np.array([(unix_times_of_averages[i], 817, avg_geog_lat[i], avg_geog_lon[i], KP_INTERPOLATED[i]) for i in range(len(KP_INTERPOLATED))])
dates = [datetime.datetime.fromtimestamp(unix_times_of_averages[i]).isoformat() for i in range(len(unix_times_of_averages))][:5000]

positions_calculated = []


for i in tqdm.tqdm(range(len(dates))):
    
    coords = {
    "x1" : points[i, 1],
    "x2" : points[i, 2],
    "x3" : points[i, 3],
    "dateTime" : dates[i]
    }

    mag_inputs = {
        "Kp" : KP_INTERPOLATED[i]
    }
        
    
    positions_calculated.append(model.make_lstar(X = coords, maginput = mag_inputs))
    
print(positions_calculated)    


100%|██████████| 5000/5000 [00:02<00:00, 2438.61it/s]

[{'Lm': [1.2107445394316128], 'MLT': [14.817562748892424], 'blocal': [17197.74789351174], 'bmin': [16976.59830799209], 'Lstar': [-1e+31], 'xj': [0.01106857414557075]}, {'Lm': [1.1992348540344557], 'MLT': [14.783224943493419], 'blocal': [17486.480081529946], 'bmin': [17480.295151099253], 'Lstar': [-1e+31], 'xj': [0.0004083554342958745]}, {'Lm': [1.196464382343141], 'MLT': [14.764160747400101], 'blocal': [17815.057086248155], 'bmin': [17592.510120583283], 'Lstar': [-1e+31], 'xj': [0.010647763705143837]}, {'Lm': [1.201606311905166], 'MLT': [14.727285785015187], 'blocal': [18489.35986713002], 'bmin': [17381.97727235698], 'Lstar': [-1e+31], 'xj': [0.054904208594685464]}, {'Lm': [1.2166448677845585], 'MLT': [14.691950339054017], 'blocal': [19346.998920262053], 'bmin': [16773.942504399423], 'Lstar': [-1e+31], 'xj': [0.1286841958964502]}, {'Lm': [1.2345656637177185], 'MLT': [14.669302992439395], 'blocal': [20118.046772384165], 'bmin': [16078.657785773214], 'Lstar': [-1e+31], 'xj': [0.203881015

In [121]:
[(model.make_lstar_output["Lm"][i], model.make_lstar_output["Lstar"][i], L_avg[i]) for i in range(25)]

[(1.2107445394316128, 1.2092224448551003, 1.2154999375343323),
 (1.1992348540344557, 1.1976020622870915, 1.2055999636650085),
 (1.196464382343141, 1.193085927283208, 1.2034999529520671),
 (1.201606311905166, 1.2005029001851386, 1.2113499641418457),
 (1.2166448677845585, 1.2169855412180512, 1.2260499894618988),
 (1.2345656637177185, 1.2338146505692127, 1.2429499626159668),
 (1.2723851037307035, 1.271942188160286, 1.2792499661445618),
 (1.3120073622866408, 1.3110894539416038, 1.3173499703407288),
 (1.3690913797825408, -1e+31, 1.3727999329566956),
 (1.4311615043740196, -1e+31, 1.4331999619801838),
 (1.5066586422733312, -1e+31, 1.5078999698162079),
 (1.6131137338538268, -1e+31, 1.6135249435901642),
 (1.7466377175991437, -1e+31, 1.7464499771595001),
 (1.8917995526800107, -1e+31, 1.8911666075388591),
 (2.0709493987428504, -1e+31, 2.0711499452590942),
 (2.330350625476344, -1e+31, 2.3295499682426453),
 (2.668494389129815, -1e+31, 2.6660749316215515),
 (3.053424145908943, 3.2458131707458566, 3.